In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms

# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('device : ', device)

In [2]:
# only for mac
device = 'mps'

In [3]:
# 0~1, -1~1
transform = transforms.Compose([
    # 0 ~ 1
    transforms.ToTensor(),
    # (0 - 0.5) / 0.5 --> -1 /  (1 -0.5)/0.5 --> 1 # -1~1
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # (mean), (std) for each channel --> (data - mean) / std
    # flip 좌우반전
    transforms.RandomHorizontalFlip(p=0.5),
])

# 0~1, -1~1
transform_test = transforms.Compose([
    # 0 ~ 1
    transforms.ToTensor(),
    # (0 - 0.5) / 0.5 --> -1 /  (1 -0.5)/0.5 --> 1 # -1~1
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # (mean), (std) for each channel --> (data - mean) / std
])

In [4]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

In [7]:
class simple_cnn(nn.Module):
  def __init__(self):
    #32/32/3
    super(simple_cnn, self).__init__()

    self.conv_layer = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=4, kernel_size=(3,3) ),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(3,3) ),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2 ),

        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3)),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3)),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # 5*5*16
    )
    # fc layer Linear

    self.fc_layer = nn.Sequential(
        nn.Dropout(0.5), 
        nn.Linear(16*5*5, 64),
        nn.ReLU(inplace=True), 
        nn.Linear(64, 32),
        nn.ReLU(inplace=True),
        nn.Linear(32, 10),        
    )

  def forward(self, x):
    o = self.conv_layer(x)
    # 5x5x16 # batch size- channel, h, w
    o = o.view(o.shape[0], -1) #numpy reshape
    o = self.fc_layer(o)

    return o


In [8]:
cnn_net = simple_cnn()

In [9]:
from torchsummary import summary
# cnn_net = cnn_net.to(device)
summary(cnn_net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 30, 30]             112
              ReLU-2            [-1, 4, 30, 30]               0
            Conv2d-3           [-1, 16, 28, 28]             592
              ReLU-4           [-1, 16, 28, 28]               0
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 16, 12, 12]           2,320
              ReLU-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 16, 10, 10]           2,320
              ReLU-9           [-1, 16, 10, 10]               0
        MaxPool2d-10             [-1, 16, 5, 5]               0
          Dropout-11                  [-1, 400]               0
           Linear-12                   [-1, 64]          25,664
             ReLU-13                   [-1, 64]               0
           Linear-14                   

In [10]:
criterion = nn.CrossEntropyLoss() #
optimizer = optim.Adam(cnn_net.parameters(), lr=0.001)

In [11]:
epochs = 10

In [13]:
cnn_net = cnn_net.to(device)
for e in range(epochs):
  for it_bat, (img, lab) in enumerate(trainloader):
    img = img.to(device)
    lab = lab.to(device)

    optimizer.zero_grad()
    outputs = cnn_net(img)

    loss = criterion(outputs, lab)
    loss.backward()

    optimizer.step()

    if (it_bat+1) % 50 == 0:
      print(f'epoch{e}, batch{it_bat+1}, loss{loss.item()}')


epoch0, batch50, loss2.3205461502075195
epoch0, batch100, loss2.185717821121216
epoch0, batch150, loss2.0120086669921875
epoch0, batch200, loss1.7870711088180542
epoch0, batch250, loss1.9749960899353027
epoch0, batch300, loss1.9124548435211182
epoch0, batch350, loss1.6788569688796997
epoch0, batch400, loss1.696638822555542
epoch0, batch450, loss1.5874279737472534
epoch0, batch500, loss1.6580092906951904
epoch1, batch50, loss1.778192162513733
epoch1, batch100, loss1.688588261604309
epoch1, batch150, loss1.4980984926223755
epoch1, batch200, loss1.7169549465179443
epoch1, batch250, loss1.6189017295837402
epoch1, batch300, loss1.5063012838363647
epoch1, batch350, loss1.516233205795288
epoch1, batch400, loss1.6039565801620483
epoch1, batch450, loss1.4284920692443848
epoch1, batch500, loss1.6124887466430664
epoch2, batch50, loss1.635170340538025
epoch2, batch100, loss1.397940993309021
epoch2, batch150, loss1.7699902057647705
epoch2, batch200, loss1.5164211988449097
epoch2, batch250, loss1.48